## A. Introduction to **Spark** and **PySpark**<br>
This notebook servers as an introductory tutorial for **Spark** and **PySpark**. PySpark is just an API in Python to work on Spark, which by default is written in Java.<br>
Also note that this notebook is referred to as the driver program that performs **Spark** operations.

Spark needs to be available first in the environment where the code is run. If a local machine is used, Spark needs to be installed. Else, if some cloud platform such as DataBricks is used, Spark is available by default.<br><br>
Once Spark is available, PySpark, a Python API to interact with the Java-based Spark, needs to be installed if not already in the environment. Then PySpark should be imported.<br><br>
However, since this notebook was created on Databricks, the Spark Context is available by default as the variable `sc`. As such, the explicit import is not necessary in order to run the basic PySpark commands as done in this notebook. But the notebook following this one might have the following cell uncommented in order to use some advanced Spark operations.

In [0]:
# from pyspark.sql import SparkSession

# create a spark session
# spark_session = SparkSession.builder.master("local[*]").getOrCreate()

# set up properties for a spark session
# spark_session.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better


# get the spark context of the spark session
# sc = spark_session.sparkContext

**Spark Context**<br>
A SparkContext used to be one of three entry points for Spark applications for Spark 1.X, namely SparkContext, SQLContext, and HiveContext. However, Spark 2.X introduced a new unified entry point called SparkSession that unifies all three previous entry points. On DataBricks, the built-in variable **sc** stores the Spark Context for any Spark application, including this notebook. In essence, a SparkContext represents a connection to a Spark cluster. It is used to perform several tasks such as create RDDs or broadcast variables on a cluster. With the new re<br><br>
Note: Databricks comes with PySpark initialized and thus we do not need to create a Spark Context manually. If this code were to run in another environment such as a local machine or a Google Colab, some extra code would have been needed to create the Spark Context. The initialized Spark Context on Databricks can be accessed using the variable **sc** as shown below.

In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 spark://10.139.64.19:7077 
 AppName 
 Databricks Shell

%md The **local** in the output above indicates that the Spark master node is the local machine of the cluster. The integer within the square brackets of **local[ INTEGER VALUE]** indicates how many cores Spark is using for the current Spark context.

## B.Creating RDDs.

#### Create a dummy data variable to create an RDD.
Typically, the data that we want to work with on Spark is taken from another source such as as a large csv file, but here we will create our own data from scratch for demo purpose. Thus taken large dataset is stored in the worker nodes of Spark cluster such that operations can be performed on them in parallel when needed.<br>

In [0]:
large_integer = 100 # 1000001 (using a smaller value 100 instead of 1000001 for demo purpose.)
large_list = list(range(0, large_integer))
len(large_list)

Out[6]: 100

This list that we just created is on the driver machine that servers as the master for other worker nodes of Spark cluster.

#### Create a Resilient Distributed Dataset (RDD).
Spark (and thus PySpark) can create Resilient Distributed Dataset (RDD) from an existing data source such that the created RDD is automatically partitioned and stored on different cluster nodes. The Spark context `sc` is the means by which this RDD can be created for the existing (typically large) data variable. The **parallelize()** of the spark context object is invoked to create an RDD.

In [0]:
large_list_rdd = sc.parallelize(large_list)
large_list_rdd

Out[7]: ParallelCollectionRDD[0] at readRDDFromInputStream at PythonRDD.scala:435

Notice that the variable does not return the actual contents of the RDD but rather a confirmation that the variable is an RDD. If we want to retrieve the actual data stored in the RDD, the `collect()` method should be called.

In [0]:
large_list_rdd.collect()

Out[10]: [0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

However, note that this `collect()` invocation is done here for demo purpose only. In practice, retrieving all the data from the clusters to the driver machine using `collect()` is not a wise thing to do since it defeats the purpose of distributing the data over the cluster. However, if we wanted to peek into what data is stored in the RDD, we could us the `take()` method to retrieve a small number of values from the RDD.

In [0]:
large_list_rdd.take(5)

Out[11]: [0, 1, 2, 3, 4]

#### Operations on RDD
Several operations can be performed on an RDD. However, the operations performed on RDDs can be categorized into two classes:<br>
- **Transformation**
  - Operations such as map, filter, join, union that are performed on RDD which return a new RDD are called transformations. Note that Spark RDDs are immutable objects and the original RDD passed to the transformation method (say, map()) remains unchanged whenever a transformation is applied. Instead, an entirely new RDD gets created after each transformation.
- **Action**
  - Operations such as count(), first(), etc. that are designed to return a new value (a summarizing value typically) are called actions. They do not return a new RDD.

Example **Transformation**: Squaring RDD elements using `map()`<br>
All elements of an RDD that contains integers can be squared using the map() function

In [0]:
squared_rdd = large_list_rdd.map(lambda x: x**2)
squared_rdd.take(5)

Out[14]: [0, 1, 4, 9, 16]

Example **Action**: Get the first element of an RDD using `count()`

In [0]:
first_element = squared_rdd.first()
first_element

Out[15]: 0